In [201]:
from bs4 import BeautifulSoup
import requests
import csv
import re

In [202]:
search_word = input("Enter a word to search :")

Enter a word to search :drone


In [203]:
base_url ="https://www.flipkart.com"
products=[]
next_page=True
trim = re.compile(r'[^\d.,]+')


In [204]:
def scrap_data(soup):
  #print(str(soup.find_all('a',class_='_2UzuFa',href=True)['href']))
  if soup.find('a',class_='_2UzuFa') != None:
    temp_prod_urls=[url['href'] for url in soup.find_all('a',class_='_2UzuFa',href=True)]
    for url in temp_prod_urls:
      single_prod=requests.get(base_url+url)
      prod_page = single_prod.text
      indi_soup = BeautifulSoup(prod_page,"html.parser")
      if indi_soup.find('span',class_="B_NuCI") != None:
        prod_name = indi_soup.find('span',class_="B_NuCI").text
        rating = indi_soup.find('span',class_="_2_R_DZ").span.text.split()[0]
        price = trim.sub('',indi_soup.find('div',class_="_30jeq3 _16Jk6d").text)
        prod_link = base_url+url
        products.append((prod_name,rating,price,prod_link))
  if soup.find('div',class_='_2kHMtA') == None:
    cards= soup.find_all('div',class_='_4ddWXP')
    for card in cards:
      prod_name = card.find('a',class_="s1Q9rs",href=True).text
      rating = card.find('span',class_="_2_R_DZ").text.replace('(','').replace(')','') if card.find('span',class_="_2_R_DZ") !=None else None
      price = trim.sub('',card.find('div',class_="_25b18c").div.text)
      prod_link = base_url+card.find('a',class_="s1Q9rs",href=True)['href']
      products.append((prod_name,rating,price,prod_link))
  else:
    cards= soup.find_all('div',class_='_2kHMtA')
    for card in cards:
        prod_name = card.find('div',class_="_4rR01T").text
        rating = card.find('span',class_="_2_R_DZ").span.span.text.split()[0] if card.find('span',class_="_2_R_DZ") !=None else None
        price = trim.sub('',card.find('div',class_="_30jeq3 _1_WHN1").text)
        prod_link = base_url+card.find('a',href=True)['href']
        products.append((prod_name,rating,price,prod_link))
    next_page_url = soup.find('a',class_="_1LKTO3",href = True)['href']
    if next_page_url != None:
        next_page=True
    else:
        next_page=False

In [205]:
curr_page=1

while next_page:
    if curr_page<3:
        url = f'https://www.flipkart.com/search?q={search_word}&page={curr_page}'
        response = requests.get(url)
        http_page = response.text
        soup = BeautifulSoup(http_page,"html.parser")
        scrap_data(soup)
        curr_page=curr_page+1
    else:
      break



In [206]:
products

[('SuperToy Mini RC Drone Without Camera - 2.4GHz 360 Flip...',
  '3',
  '2,349',
  'https://www.flipkart.com/supertoy-mini-rc-drone-without-camera-2-4ghz-360-flip-headless-mode-multicolo/p/itmcee9b175d5a7f?pid=DNEGBTJDZBURHRVY&lid=LSTDNEGBTJDZBURHRVYKMWXWE&marketplace=FLIPKART&q=drone&store=search.flipkart.com&srno=s_1_1&otracker=search&fm=organic&iid=7041b900-d103-40cf-9aca-1792a1d71c1a.DNEGBTJDZBURHRVY.SEARCH&ppt=None&ppn=None&ssid=vkz7ulujgg0000001649055513643&qH=030e39a1278a1882'),
 ('Toyify Foldable Camera Drone With Mobile App / Remote C...',
  None,
  '6,948',
  'https://www.flipkart.com/toyify-foldable-camera-drone-mobile-app-remote-control-480p-basic-10-min-flying-time/p/itm8834f9368b642?pid=DNEG72G9YPWCDAK7&lid=LSTDNEG72G9YPWCDAK7B1LZBZ&marketplace=FLIPKART&q=drone&store=search.flipkart.com&srno=s_1_2&otracker=search&fm=organic&iid=7041b900-d103-40cf-9aca-1792a1d71c1a.DNEG72G9YPWCDAK7.SEARCH&ppt=None&ppn=None&ssid=vkz7ulujgg0000001649055513643&qH=030e39a1278a1882'),
 ('Amita

In [207]:

len(products)

80

In [208]:
fields = ['prod_name','ratings','price','prod_url']
with open(f'{search_word}.csv', 'w') as f:
      
    # using csv.writer method from CSV package
    write = csv.writer(f)
      
    write.writerow(fields)
    write.writerows(products)